## Merchandise Prices Prediction

Object: To build an algorithm that automatically suggests the right product prices. <br>
The datasets contains 29,999 kinds of products, each with 7 features of one kind of products.

Features:
    - ID
    - Name
    - Item Condition
    - Category
    - Brand Name 
    - Price 
    - Shipping

In [ ]:
Outlines:
    - Exploring data
    - Impelement LightGMB and Ridge regression model, take result based on 40/60 from both models respectively 
    - Feature extraction:
        - 

In [1]:
import pandas as pd

In [2]:
train = pd.read_csv('../M_Project/train.tsv',delimiter='\t',encoding='utf-8')
test = pd.read_csv('../M_Project/test.tsv',delimiter='\t',encoding='utf-8')


In [3]:
train.dtypes

train_id               int64
name                  object
item_condition_id      int64
category_name         object
brand_name            object
price                float64
shipping               int64
item_description      object
dtype: object

### Exploring Data sets

In [4]:
train.head()

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description
0,0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,NaN,10.0,1,No description yet
1,1,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,52.0,0,This keyboard is in great condition and works ...
2,2,AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,Target,10.0,1,Adorable top with a hint of lace and a key hol...
3,3,Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,NaN,35.0,1,New with tags. Leather horses. Retail for [rm]...
4,4,24K GOLD plated rose,1,Women/Jewelry/Necklaces,NaN,44.0,0,Complete with certificate of authenticity


In [5]:
train.shape

(29999, 8)

In [6]:
test.shape

(29999, 7)

In [7]:
train.describe()

,train_id,item_condition_id,price,shipping
count,29999.000000,29999.000000,29999.000000,29999.000000
mean,14999.000000,1.906164,26.953582,0.447715
std,8660.109699,0.906798,40.394998,0.497267
min,0.000000,1.000000,0.000000,0.000000
25%,7499.500000,1.000000,10.000000,0.000000
50%,14999.000000,2.000000,17.000000,0.000000
75%,22498.500000,3.000000,29.000000,1.000000
max,29998.000000,5.000000,1506.000000,1.000000


In [8]:
train_col = list(train)
train_col

['train_id',
 'name',
 'item_condition_id',
 'category_name',
 'brand_name',
 'price',
 'shipping',
 'item_description']

In [9]:
test_col = list(test)
test_col

['test_id',
 'name',
 'item_condition_id',
 'category_name',
 'brand_name',
 'shipping',
 'item_description']

In [10]:
train_col[0] = 'id'
test_col[0] = 'id'

In [11]:
train.columns = train_col
test.columns = test_col

In [12]:
train.columns.difference(test.columns)

Index(['price'], dtype='object')

In [13]:
import gc
import time
import pandas as pd
import numpy as np

In [14]:
from scipy.sparse import csr_matrix, hstack

In [15]:
from sklearn.linear_model import Ridge
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split, cross_val_score

In [16]:
import lightgbm as lgb

In [17]:
NUM_BRANDS = 4000
NUM_CATEGORIES = 1000
NAME_MIN_DF = 10
MAX_FEATURES_ITEM_DESCRIPTION = 50000

In [18]:


What is the variation between prices?
How good the products are?
How’s the shipping conditions?
What are the premium brands?
Dose premium brands sold at higher prices?
How many categories?
Does prices very between categories?


Object `prices` not found.
Object `are` not found.
Object `conditions` not found.
Object `brands` not found.
Object `prices` not found.
Object `categories` not found.
Object `categories` not found.


In [19]:
Does prices very between categories

SyntaxError: invalid syntax (<ipython-input-19-bf0828af4642>, line 1)

#### Transform missing data in categotical columns

In [20]:
def handle_missing_inplace(dataset):
    dataset['category_name'].fillna(value='missing', inplace=True)
    dataset['brand_name'].fillna(value='missing', inplace=True)
    dataset['item_description'].fillna(value='missing', inplace=True)

#### Trimming dataset

In [21]:
def cutting(dataset):
    pop_brand = dataset['brand_name'].value_counts().loc[lambda x: x.index != 'missing'].index[:NUM_BRANDS]
    dataset.loc[~dataset['brand_name'].isin(pop_brand), 'brand_name'] = 'missing'
    pop_category = dataset['category_name'].value_counts().loc[lambda x: x.index != 'missing'].index[:NUM_BRANDS]
 

In [22]:
def to_categorical(dataset):
    dataset['category_name'] = dataset['category_name'].astype('category')
    dataset['brand_name'] = dataset['brand_name'].astype('category')
    dataset['item_condition_id'] = dataset['item_condition_id'].astype('category')


#### Model takes result based on 40/60 from LightGBM and Ridge models respectively 

In [28]:


def mixed_modle():
    start_time = time.time()
    
    train = pd.read_table('../M_Project/train.tsv', engine='c')
    test = pd.read_table('../M_Project/test.tsv', engine='c')
    print('[{}] Finished to load data'.format(time.time() - start_time))
    print('Train shape: ', train.shape)
    print('Test shape: ', test.shape)

    nrow_train = train.shape[0]
    y = np.log1p(train["price"])
    merge: pd.DataFrame = pd.concat([train, test])
    submission: pd.DataFrame = test[['test_id']]
        
    del train
    del test
    gc.collect()
    
    # Feature extraction
    handle_missing_inplace(merge)
    print('[{}] Finished handling missing data'.format(time.time() - start_time))
    
    cutting(merge)
    print('[{}] Finished cutting data'.format(time.time() - start_time))
    
    to_categorical(merge)
    print('[{}] Finished converting data'.format(time.time() - start_time))

    cv = CountVectorizer(min_df=NAME_MIN_DF)
    X_name = cv.fit_transform(merge['name'])
    print('[{}] Finished counting vectorizer name'.format(time.time() - start_time))

    cv = CountVectorizer()
    X_category = cv.fit_transform(merge['category_name'])
    print('[{}] Finished counting vectorizer category_name'.format(time.time() - start_time))
    
    tv = TfidfVectorizer(max_features=MAX_FEATURES_ITEM_DESCRIPTION,
                         ngram_range=(1, 3),
                         stop_words='english')
    X_description = tv.fit_transform(merge['item_description'])
    print('[{}] Finished TPIDF vectorizing'.format(time.time() - start_time))
    
    lb = LabelBinarizer(sparse_output=True)
    X_brand = lb.fit_transform(merge['brand_name'])
    print('[{}] Finished label-binarizing brand_names'.format(time.time() - start_time))
    
    X_dummies = csr_matrix(pd.get_dummies(merge[['item_condition_id', 'shipping']], sparse=True).values)
    print('[{}] Finished getting dummies on item_condition_id and shipping'.format(time.time() - start_time))
    
    sparse_merge = hstack((X_dummies, X_description, X_brand, X_category, X_name)).tocsr()
    print('[{}] Finished creating sparse merge'.format(time.time() - start_time))
    
    X = sparse_merge[:nrow_train]
    X_test = sparse_merge[nrow_train:]
    
    #X_train, X_test, y_train, y_test = train_test_split(train_set, y_train_val, test_size=0.2, random_state=123)
    d_train = lgb.Dataset(X, label=y)
    #d_valid = lgb.Dataset(valid_X, label=valid_y)
    
    params = {
        'learning_rate': 0.75,
        'application': 'regression',
        'max_depth': 3,
        'num_leaves': 100,
        'verbosity': -1,
        'metric': 'RMSE',
    }
    
    
    model = lgb.train(params, train_set=d_train, num_boost_round=3200, verbose_eval=100) 
    print('[{}] Finished training LightGBM'.format(time.time() - start_time))
    preds = 0.6 * model.predict(X_test)
    print('[{}] Finished predicting by LightGBM'.format(time.time() - start_time))
    model = Ridge(solver="sag", fit_intercept=True, random_state=205)
    model.fit(X, y)
    print('[{}] Finished training ridge'.format(time.time() - start_time))
    preds += 0.4 * model.predict(X=X_test)
    print('[{}] Finished predicting by ridge'.format(time.time() - start_time))
    
    
    

In [29]:
mixed_modle()

[0.3394200801849365] Finished to load data
Train shape:  (29999, 8)
Test shape:  (29999, 7)
[0.41411614418029785] Finished handling missing data
[0.46460700035095215] Finished cutting data
[0.48879504203796387] Finished converting data
[1.0793750286102295] Finished counting vectorizer name
[1.5937409400939941] Finished counting vectorizer category_name
[11.58866286277771] Finished TPIDF vectorizing
[13.750045776367188] Finished label-binarizing brand_names
[14.00936222076416] Finished getting dummies on item_condition_id and shipping
[14.17724895477295] Finished creating sparse merge
[60.461758852005005] Finished training LightGBM
[66.60012888908386] Finished predicting by LightGBM
[69.83184885978699] Finished training ridge
[69.83708786964417] Finished predicting by ridge


In [ ]:
def ridge_modle():
    start_time = time.time()
    
    train = pd.read_table('train.tsv', engine='c')
    test = pd.read_table('test.tsv', engine='c')
    print('[{}] Finished to load data'.format(time.time() - start_time))
    print('Train shape: ', train.shape)
    print('Test shape: ', test.shape)

    nrow_train = train.shape[0]
    y = np.log1p(train["price"])
    merge: pd.DataFrame = pd.concat([train, test])
    submission: pd.DataFrame = test[['test_id']]
        
    del train
    del test
    gc.collect()
    
    # Feature extraction
    handle_missing_inplace(merge)
    print('[{}] Finished handling missing data'.format(time.time() - start_time))
    
    cutting(merge)
    print('[{}] Finished cutting data'.format(time.time() - start_time))
    
    to_categorical(merge)
    print('[{}] Finished converting data'.format(time.time() - start_time))

    cv = CountVectorizer(min_df=NAME_MIN_DF)
    
    
    X_name = cv.fit_transform(merge['name'])
    print('[{}] Finished counting vectorizer name'.format(time.time() - start_time))

    cv = CountVectorizer()
    X_category = cv.fit_transform(merge['category_name'])
    print('[{}] Finished counting vectorizer category_name'.format(time.time() - start_time))
    
    tv = TfidfVectorizer(max_features=MAX_FEATURES_ITEM_DESCRIPTION,
                         ngram_range=(1, 3),
                         stop_words='english')
    X_description = tv.fit_transform(merge['item_description'])
    print('[{}] Finished TPIDF vectorizing'.format(time.time() - start_time))
    
    lb = LabelBinarizer(sparse_output=True)
    X_brand = lb.fit_transform(merge['brand_name'])
    print('[{}] Finished label-binarizing brand_names'.format(time.time() - start_time))
    
    # Convert categorical variables into dummy variables
    X_dummies = csr_matrix(pd.get_dummies(merge[['item_condition_id', 'shipping']], sparse=True).values)
    print('[{}] Finished getting dummies on item_condition_id and shipping'.format(time.time() - start_time))
    
    #
    sparse_merge = hstack((X_dummies, X_description, X_brand, X_category, X_name)).tocsr()
    print('[{}] Finished creating sparse merge'.format(time.time() - start_time))
    
    X = sparse_merge[:nrow_train]
    X_test = sparse_merge[nrow_train:]
    
    #X_train, X_test, y_train, y_test = train_test_split(train_set, y_train_val, test_size=0.2, random_state=123)
    d_train = lgb.Dataset(X, label=y)
    #d_valid = lgb.Dataset(valid_X, label=valid_y)
    
    params = {
        'learning_rate': 0.75,
        'application': 'regression',
        'max_depth': 3,
        'num_leaves': 100,
        'verbosity': -1,
        'metric': 'RMSE',
    }
    
    
    model = Ridge(solver="sag", fit_intercept=True, random_state=205)
    model.fit(X, y)
    print('[{}] Finished to train ridge'.format(time.time() - start_time))
    preds = model.predict(X=X_test)
    print('[{}] Finished to predict ridge'.format(time.time() - start_time))
    
    

In [214]:
#if __name__ == '__main__':
#    main()